In [202]:
import pandas as pd 
import numpy as np

# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [203]:
import pandas as pd

In [211]:
# your code here
def relevant_years(df):
    return df[(df['DATE'] >= '1993-01-01') & (df['DATE'] <= '2023-12-31')].copy()

df = pd.read_csv('BR.csv')
df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'], dayfirst=False) 
df = df.rename(columns={'OBS_VALUE': 'Current_Rate'})
df['Year'] = df['TIME_PERIOD'].dt.year
df['Month'] = df['TIME_PERIOD'].dt.month
monthly_data = df.groupby(['Year', 'Month'])['Current_Rate'].last().reset_index()
monthly_data['Change_From_Last_Month'] = monthly_data['Current_Rate'].diff()
data_of_israel_intrest = monthly_data[['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]

# Load the US interest rate data
df = pd.read_csv('FEDFUNDS.csv')
df['DATE'] = pd.to_datetime(df['DATE'])  
df = df.rename(columns={'FEDFUNDS': 'Current_Rate'})
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month
monthly_data = df.groupby(['Year', 'Month'])['Current_Rate'].last().reset_index()
monthly_data['Change_From_Last_Month'] = monthly_data['Current_Rate'].diff()
data_of_usa_intrest = monthly_data[['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]

# Load the USD/ILS exchange rate data
df = pd.read_csv('USD_ILS.csv')
df['DATE'] = pd.to_datetime(df['תאריך'], dayfirst=True)  
df = df.rename(columns={'שער': 'Current_Rate'})
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month
monthly_data = df.groupby(['Year', 'Month'])['Current_Rate'].last().reset_index()
monthly_data['Change_From_Last_Month'] = monthly_data['Current_Rate'].diff()
data_usd_ils_ex = monthly_data[['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]


## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [35]:
# A - 1
# your code here
data_of_usa_intrest.head(5)


,Year,Month,Current_Rate,Change_From_Last_Month
0,1993,1,3.02,NaN
1,1993,2,3.03,0.01
2,1993,3,3.07,0.04
3,1993,4,2.96,-0.11
4,1993,5,3.00,0.04


In [127]:
# A - 2
# your code here

average_israel_rate = data_of_israel_intrest['Current_Rate'].mean()
average_israel_rate

4.879444444444445

In [125]:
# A - 3
# your code here

max_change_row_month = data_usd_ils_ex.loc[data_usd_ils_ex['Change_From_Last_Month'].abs().idxmax()]


max_change_row_year = data_usd_ils_ex.groupby('Year')['Change_From_Last_Month'].sum().reset_index()
max_change_row_year = max_change_row_year.rename(columns={'Change_From_Last_Month': 'most_change_year'})
max_change_row_year = max_change_row_year.loc[max_change_row_year['most_change_year'].idxmax()]
max_change_row_year

Year                2014.0000
most_change_year       0.4254
Name: 2, dtype: float64

In [144]:
# A - 4
# your code here


decreased_months_count = data_usd_ils_ex[data_usd_ils_ex['Change_From_Last_Month'] < 0]
decreased_months_count

,Year,Month,Current_Rate,Change_From_Last_Month
6,2012,9,3.9198,-0.1068
7,2012,10,3.8836,-0.0362
8,2012,11,3.8153,-0.0683
9,2012,12,3.7316,-0.0837
10,2013,1,3.7098,-0.0218
...,...,...,...,...
122,2022,5,3.3221,-0.0191
124,2022,7,3.3968,-0.0981
125,2022,8,3.3283,-0.0685
127,2022,10,3.5214,-0.0414


In [158]:
# A - 5
# your code here

data_of_israel_intrest['Change_Percentage'] = data_of_israel_intrest['Current_Rate'].pct_change() * 100
data_of_israel_intrest['Change_Percentage']
data_of_israel_intrest.head(8)

,Year,Month,Current_Rate,Change_From_Last_Month,Change_Percentage
0,1994,1,10.5,NaN,NaN
1,1994,2,10.5,0.0,0.000000
2,1994,3,10.5,0.0,0.000000
3,1994,4,10.5,0.0,0.000000
4,1994,5,11.5,1.0,9.523810
5,1994,6,12.0,0.5,4.347826
6,1994,7,12.5,0.5,4.166667
7,1994,8,12.5,0.0,0.000000


## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [166]:
# B - 1
# your code here


merged_df = pd.merge(data_of_israel_intrest, data_of_usa_intrest, on= ['Year', 'Month'], suffixes=('_Israel', '_US'))
merged_df

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US
0,1994,1,10.50,NaN,NaN,3.05,0.09
1,1994,2,10.50,0.0,0.00000,3.25,0.20
2,1994,3,10.50,0.0,0.00000,3.34,0.09
3,1994,4,10.50,0.0,0.00000,3.56,0.22
4,1994,5,11.50,1.0,9.52381,4.01,0.45
...,...,...,...,...,...,...,...
355,2023,8,4.75,0.0,0.00000,5.33,0.21
356,2023,9,4.75,0.0,0.00000,5.33,0.00
357,2023,10,4.75,0.0,0.00000,5.33,0.00
358,2023,11,4.75,0.0,0.00000,5.33,0.00


In [225]:
# B - 2
# your code here


merged_df['Rate_Difference'] = merged_df['Current_Rate_Israel'] - merged_df['Current_Rate_US']

merged_df

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US,Rate_Difference,Rate_Comparison,Rate_Percentage_Difference
0,1994,1,10.50,NaN,NaN,3.05,0.09,7.45,IH,244.262295
1,1994,2,10.50,0.0,0.00000,3.25,0.20,7.25,IH,223.076923
2,1994,3,10.50,0.0,0.00000,3.34,0.09,7.16,IH,214.371257
3,1994,4,10.50,0.0,0.00000,3.56,0.22,6.94,IH,194.943820
4,1994,5,11.50,1.0,9.52381,4.01,0.45,7.49,IH,186.783042
...,...,...,...,...,...,...,...,...,...,...
355,2023,8,4.75,0.0,0.00000,5.33,0.21,-0.58,UH,-10.881801
356,2023,9,4.75,0.0,0.00000,5.33,0.00,-0.58,UH,-10.881801
357,2023,10,4.75,0.0,0.00000,5.33,0.00,-0.58,UH,-10.881801
358,2023,11,4.75,0.0,0.00000,5.33,0.00,-0.58,UH,-10.881801


In [197]:
# B - 3
# your code here


merged_df['Rate_Percentage_Difference'] = (merged_df['Rate_Difference'] / merged_df['Current_Rate_US']) * 100

cond = [
    merged_df['Rate_Percentage_Difference'] >= 1,
    merged_df['Rate_Percentage_Difference'] <= -1
    
]

choices = ["IH", "UH"]


merged_df["Rate_Comparison"] = np.select(cond, choices, default="N")

merged_df[merged_df["Rate_Comparison"] == 'N']


In [226]:
# B - 4
# your code here


final_merged_data = pd.merge(merged_data, data_usd_ils_ex, on=['Year', 'Month'], suffixes=('_Interest', '_Exchange'))


final_merged_data

,Year,Month,Current_Rate_Israel,Change_From_Last_Month_Israel,Change_Percentage,Current_Rate_US,Change_From_Last_Month_US,Rate_Difference,Current_Rate,Change_From_Last_Month
0,2012,3,2.50,0.00,0.000000,0.13,0.03,2.37,3.7500,NaN
1,2012,4,2.50,0.00,0.000000,0.14,0.01,2.36,3.7252,-0.0248
2,2012,5,2.50,0.00,0.000000,0.16,0.02,2.34,3.7774,0.0522
3,2012,6,2.25,-0.25,-10.000000,0.16,0.00,2.09,3.9091,0.1317
4,2012,7,2.25,0.00,0.000000,0.16,0.00,2.09,3.9234,0.0143
...,...,...,...,...,...,...,...,...,...,...
125,2022,8,2.00,0.75,60.000000,2.33,0.65,-0.33,3.3680,-0.1557
126,2022,9,2.00,0.00,0.000000,2.56,0.23,-0.56,3.3786,0.0106
127,2022,10,2.75,0.75,37.500000,3.08,0.52,-0.33,3.5363,0.1577
128,2022,11,3.25,0.50,18.181818,3.78,0.70,-0.53,3.5313,-0.0050


In [231]:
# B - 5
# your code here
correlation = final_merged_data[['Rate_Difference', 'Current_Rate']].corr()



"""
הקשר החיובי המשמעותו היא שכאשר יש הבדל ריביות חיובי (כלומר, כאשר הריבית בישראל גבוהה יותר מאשר בארה"ב), שער הדולר נוטה לעלות. כלומר, ככל שהריבית בישראל גבוהה יותר לעומת ארה"ב, הדולר מתחזק ביחס לשקל, כלומר שער הדולר/שקל עולה.

עם זאת, כיוון שהקשר הוא חלש (כפי שרואים ערך הקורלציה של 0.186), ניתן להסיק שהשפעת ההבדל בריביות על שער הדולר/שקל אינה מאוד חזקה או ישירה. ישנם גורמים נוספים בשוק המטבעות שעשויים להשפיע על שער הדולר/שקל, מעבר להבדל בריביות בין המדינות.
"""
correlation

,Rate_Difference,Current_Rate
Rate_Difference,1.000000,0.183292
Current_Rate,0.183292,1.000000


## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [233]:

correlation2 = final_merged_data['Change_From_Last_Month_US'].corr(final_merged_data['Change_From_Last_Month'])
correlation2



0.00295310885232544

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here